In [ ]:
# Example of utils.videowriter 
from importlib import reload

import utils.videowriter
reload(utils.videowriter)
from utils.videowriter import VideoWriter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time as ti
from matplotlib import cm
from tqdm import tqdm

In [ ]:
def macroscopic(fin, nx, ny, v):
    rho = np.sum(fin,axis=0)
    u = np.zeros((2,nx,ny))
    for i in range(9):
        u[0,:,:] += v[i,0]*fin[i,:,:]
        u[1,:,:] += v[i,1]*fin[i,:,:]
    u /= rho
    return rho, u

In [ ]:
def equilibrium(rho, u, v, t, nx, ny):
    usqr = (3/2)*(u[0]**2+u[1]**2)
    feq = np.zeros((9,nx,ny))
    for i in range(9):
        cu = 3*(v[i,0]*u[0,:,:] + v[i,1]*u[1,:,:])
        feq[i,:,:] = rho*t[i]*(1+cu+0.5*cu**2-usqr)
    return feq

In [ ]:
def obstacle_fun(cx, cy, r):
    def inner(x, y):
        return (x-cx)**2+(y-cy)**2<r**2
    return inner


In [ ]:
def inivel( uLB, ly):
    def inner(d,x,y):
        return (1-d) * uLB * (1+1e-4*np.sin(y/ly*2*np.pi))
    return inner

# inlet_vel = 0.1
# ly = 1

# np.fromfunction(inivel(0.1, 1), (2, 10, 10))

In [ ]:
# d=2
# uLB=0.04
# ly=99
# y=99

# (1-d) - (1+1e-4*np.sin(y/ly*2*np.pi))

In [ ]:
Re = 10.0                  # Reynolds number
#------------------------------------------------------------------------------
maxIter = 1000
nx,ny = 15,15             # Domain dimensions
ly = ny-1
uLB = 0.04                  # Inlet velocity NON PHYSICAL??
cx,cy,r = nx//4,ny//2,ny/9  # cylinder coordinates and radius (as integers)
nulb = uLB*r/Re             # Viscosity
omega = True / (3*nulb+0.5)    # Relaxation parameter

# lattice velocities
v = np.array([ 
            [1,1],
            [1,0],
            [1,-1],
            [0,1],
            [0,0],
            [0,-1],
            [-1,1],
            [-1,0],
            [-1,-1]
            ])

# weights
t = np.array([ 
            1/36,
            1/9,
            1/36,
            1/9,
            4/9,
            1/9,
            1/36,
            1/9,
            1/36
            ])

col_0 = np.array([0,1,2])
col_1 = np.array([3,4,5])
col_2 = np.array([6,7,8])

# instantiate the cylindrical obstacle
# obstacle = np.fromfunction(obstacle_fun(cx,cy,r),(nx, ny))
# Example grid mask for obstacles (tunnel walls)

obstacle = np.zeros((nx, ny))
# Sets the boundary of the grid to be an obstacle
obstacle[0, :] = True
obstacle[-1, :] = True
obstacle[:, 0] = True
obstacle[:, -1] = True
# Creates a narrow tunnel going through the middle of the grid
obstacle[nx//2-2:nx//2+2, :] = True
# hollows the tunnel
obstacle[nx//2-1:nx//2+1, 1:-1] = False
# creates an opening in the tunnel on the top right and bottom left
obstacle[nx//2-2, -2] = False
obstacle[nx//2+1, 1] = False

obstacle = obstacle.astype(bool)

# initial velocity profile
# vel = np.fromfunction(inivel(uLB,ly),(2,nx,ny))
vel = np.zeros((2,nx,ny))

# initialize fin to equilibirum (rho = 1)
fin = equilibrium(1,vel,v,t,nx,ny) * 0 

# Sets the density fields in the middle to twice the density
fin[0, nx//2-1:nx//2+1, :] = 2


# Displays the environment, including the velocity, density, and obstacles as different colors
def display(fin, vel, obstacle, nx, ny):
    rho, u = macroscopic(fin, nx, ny, v)
    plt.figure(figsize=(10,10))
    plt.imshow(np.sqrt(u[0]**2+u[1]**2).T, cmap=cm.Reds, interpolation='nearest')
    plt.imshow(obstacle.T, cmap=cm.Greys, interpolation='nearest', alpha=0.5)
    plt.axis('off')
    # creates legend
    plt.legend(handles=[plt.Line2D([0], [0], color='red', lw=4, label='Velocity'),
                        plt.Line2D([0], [0], color='grey', lw=4, label='Obstacle')],
                loc='lower right')
    
    plt.show()


  
display(fin, vel, obstacle, nx, ny)


In [ ]:
#==============================================================================
#   Time-Stepping
#==============================================================================
t0 = ti()
vid = VideoWriter("./fluid.mp4", fps=33)

wave_cell = 1

max_pressure_per_time_step = []

for time in tqdm(range(maxIter)):
    # outflow boundary condition (right side) NEUMANN BC! No gradient
    # fin[col_2,-1,:] = fin[col_2,-2,:]
    # 

    # records the maximum pressure per time step
    max_pressure_per_time_step.append(np.max(fin[1,-1,:]))

    # Multiplies cell density by 1.5 along the pipe, once cell per 10 time steps
    if time % 10 == 0:
        fin[0, nx//2-1:nx//2+1, wave_cell] *= 1.5
        wave_cell += 1
        if wave_cell == ny-1:
            wave_cell = 1

    # compute macroscopic variables
    rho,u = macroscopic(fin,nx,ny,v)

    # inlet boundary condition (left wall)
    # u[:,0,:] = vel[:,0,:]
    # rho[0,:] = 1/(1-u[0,0,:])*( np.sum(fin[col_1,0,:], axis = 0)+
    #                             2*np.sum(fin[col_2,0,:], axis = 0))

    feq = equilibrium(rho,u,v,t,nx,ny)
    fin[col_0,0,:] = feq[col_0,0,:] + fin[col_2,0,:]-feq[col_2,0,:]

    # Collide
    fout = fin - omega*(fin-feq)

    # bounceback
    for i in range(9):
        fout[i,obstacle] = fin[8-i,obstacle]

    # stream
    for i in range(9):
        # be careful with this -> numpy.roll cycles through an array by an axis
        # and the last element becomes the first. this implements a periodic
        # boundary in a very compact syntax, but needs to be reworked for other
        # implementations
        fin[i,:,:] = np.roll(  
                          np.roll(
                                fout[i,:,:], v[i,0], axis = 0
                               ),
                          v[i,1], axis = 1 
                          )

    # Output an image every 100 iterations
    if time%5 == 0:
        # vid.add_grid(np.sqrt(u[0]**2+u[1]**2).T, cmap = cm.Reds)
        # sets the left corner rho to a pressure of 0.5 and the right corner to a rho of 0 to help the colormapping, stores values to be resotred
        left_corner = [1,0,0]
        right_corner = fin[1,-1,0]
        fin[1,0,0] = 10
        fin[1,-1,0] = 0
        vid.add_grid(fin)
        fin[1,0,0] = left_corner
        fin[1,-1,0] = right_corner
        # resets the left corner to a pressure of 0
    # if (time%100 == 0):
    #     plt.clf()
    #     plt.imshow(np.sqrt(u[0]**2+u[1]**2).T, cmap = cm.Reds)
    #     plt.savefig("vel{0:03d}.png".format(time//100))
tf = ti() - t0
print("time to execute = ",tf)
vid.close()

# Plots the maximum pressure per time step
plt.plot(max_pressure_per_time_step)

In [ ]:
fin.shape

In [ ]:
# Interactive Matplotlib for simulating


# Old

In [ ]:
import numpy as np

# Horiz velocity of every cell in the grid, these are placed "in the middle of the vertical lines" of the grid
grid_width = 100
grid_height = 100
horizonal_velocities = np.zeros((grid_width, grid_height))
vertical_velocities = np.zeros((grid_width, grid_height))

# Example grid mask for sources of pressure
sources = np.zeros((grid_width, grid_height))
sources[48:51, 48:51] = 1

# Example grid mask for obstacles (tunnel walls)
obstacles = np.zeros((grid_width, grid_height))
# Sets the boundary of the grid to be an obstacle
obstacles[0, :] = True
obstacles[-1, :] = True
obstacles[:, 0] = True
obstacles[:, -1] = True
# Sets a tunnel in the middle of the grid
obstacles[40, :] = True
obstacles[60, :] = True
obstacles[40, 40:60] = 0


# Shows everything
import matplotlib.pyplot as plt
plt.imshow(sources)
plt.show()
plt.imshow(obstacles)
plt.show()


## Setting up the environment

## Modify velocity values

In [ ]:
def flow_sources(horizontal_velocities, vertical_velocities, sources, velocity, dt):
    """
    Adds flow to the grid based on the sources,
    velocities area evenly added to particles on the edges of the source cells
    """
    delta_velocity = sources * velocity * dt
    horizontal_velocities        -= delta_velocity
    vertical_velocities          -= delta_velocity
    horizontal_velocities[:, 1:] += delta_velocity
    vertical_velocities[1:, :]   += delta_velocity

## Projection (making fluid incompressible)

#### Divergence (Total Outflow)

In [ ]:
def calc_divergence(horizontal_velocities, vertical_velocities):
    """
    Calculates the outflow of every cell
    """
    return (horizontal_velocities[:, 1:] - horizontal_velocities) + (vertical_velocities[1:, :] - vertical_velocities)

def get_n_blocked_neighbors(obstacles):
    return np.roll(obstacles, 1, axis=0) + np.roll(obstacles, -1, axis=0) + np.roll(obstacles, 1, axis=1) + np.roll(obstacles, -1, axis=1)
    # TODO: INCOMPLETE, stopped here, goodnight

def fix_divergence(horizontal_velocities, vertical_velocities, divergence, n_blocked_neighbors):
    """
    Fixes the divergence of the grid by adding velocity to the cells such that divergence is 0
    """
    # The amount of velocity to add to each cell
    delta_velocity = divergence / (4 - n_blocked_neighbors)
    # Add the velocity to the cells
    horizontal_velocities[:, 1:] -= delta_velocity
    vertical_velocities[1:, :]   -= delta_velocity
    horizontal_velocities        += delta_velocity
    vertical_velocities          += delta_velocity

In [ ]:
# displays n_blocked_neighbors
import matplotlib.pyplot as plt
obstacles [50,50] = 1
plt.imshow(get_n_blocked_neighbors(obstacles))

### The code for applying pressure.

Here mp_p0 is the array that stores the density (which is equivalent to the pressure, so I actually refer to it as pressure in the code).

The arrays mp_xv1 and mp_yv1 store the x and y components of the velocity field.

The function Cell(x,y) returns a cell index for a given set of x and y coordinates.

The loop simply iterates over all horizontal and vertical pairs of cells, finds the difference in pressure, scales it by a constant (also scaled by time) and adds it to both cells.

```C
for (int x = 0; x < m_w-1; x++) {
    for (int y = 0; y < m_h-1; y++) {
        int cell = Cell(x,y);
        float force_x =  mp_p0[cell] - mp_p0[cell+1];
        float force_y =  mp_p0[cell] - mp_p0[cell+m_w];
        mp_xv1[cell]     +=  a * force_x;
        mp_xv1[cell+1]   +=  a * force_x;
        mp_yv1[cell]     +=  a * force_y;
        mp_yv1[cell+m_w] +=  a * force_y;
    }
}
```